# Bulk Access to OpenTopography's Global Datasets 
## Leveraging the Power of COGs

[OpenTopography](https://opentopography.org/) strives to provide a variety of access methods to our datasets to address the varying needs of our users. While most users access data through the web map interface, others prefer to programmatically access data from our cloud-based storage. In this tutorial we will explore a couple of different methods of accessing data over the Grand Canyon from the cloud-based [NASA Shuttle Radar Topography Mission (SRTM) catalog in OpenTopography](https://doi.org/10.5069/G9445JDF).  We will also demonstrate how we can use open-source, command line tools to programmatically access data without the need for a web-map interface. In addition, this tutorial should highlight the power of [Cloud Optimized Geotiffs (COGs).](https://www.cogeo.org/) and how they can be used to reduce download filesizes as well as increase data access speeds.

COGs are quickly becoming a standard format for hosting raster data in the cloud due to its optimized file structure to enable faster access and subsetting. In order to take advantage of this benefit, [OpenTopography](https://opentopography.org/) has recently converted its entire global dataset collection to COGs. With the COG format, programs and web requests can read the top-level header and quickly extract data for a region of interest as opposed to having to read the entire file.  This drastically reduces file read-time as well as reducing the number of http GET requests to access the data.  For a more detailed discussion on COGs and their benefits see the resources section at the bottom of this page.  

For this tutorial, it is recommended to set up a conda environment with GDAL 3.2 and AWS CLI tools installed:

`# Create a conda environment called, 'COGS', and install GDAL v 3.2:
 conda create -n COGS -c conda-forge gdal==3.2.0`

 `#Activate your new environment
 conda activate COGS` 
 
 `#install AWS command line tools in your environment
 conda install -c conda-forge awscli`

 Below is a Google Earth image of a section of the Grand Canyon, AZ where we will extract data:

![GrandCanyon](img/GC_Basemap.png)

---
The [SRTM catalog in OpenTopography](https://doi.org/10.5069/G9445JDF) is organized into tiled COGs where each tile is named after its lower left corner (e.g. N35W112.tif is the tile that has its South-West corner at 35 Degrees North, 112 degrees West). 

For this exercise we will gather data around the Grand Canyon with a lat/lon bounds of [-114, 35, -111, 37].  

Using this Area of Interest (AOI) information we can grab the tiles that cover this region.  OpenTopography users can access all hosted data via AWS Command Line Tools.  So, knowing the lat/lon bounds of our area of interest, we can download just the tiles for this region using [aws command line tools](https://anaconda.org/conda-forge/awscli).  

It is useful to incorporate wildcards to limit the data you need to download.  The example belows downloads all files that match: N35W112.tif, N35W113.tif, N35W114.tif, N36W114.tif, N36W112.tif, & N36W113.tif 

In [3]:
!aws s3 cp s3://raster/SRTM_GL1/SRTM_GL1_srtm/ . --recursive --exclude "*" --include "N3[56]W11[234].tif" --endpoint-url https://opentopography.s3.sdsc.edu --no-sign-request

download: s3://raster/SRTM_GL1/SRTM_GL1_srtm/N35W112.tif to ./N35W112.tif             
download: s3://raster/SRTM_GL1/SRTM_GL1_srtm/N35W113.tif to ./N35W113.tif          
download: s3://raster/SRTM_GL1/SRTM_GL1_srtm/N36W112.tif to ./N36W112.tif          
download: s3://raster/SRTM_GL1/SRTM_GL1_srtm/N35W114.tif to ./N35W114.tif
download: s3://raster/SRTM_GL1/SRTM_GL1_srtm/N36W113.tif to ./N36W113.tif
download: s3://raster/SRTM_GL1/SRTM_GL1_srtm/N36W114.tif to ./N36W114.tif


---
Plotting the downloaded geotiffs in QGIS on top of Googe Earth imagery:

![Basemap with SRTM](img/GC_wSRTMTiles.png)

Downloading the data in this manner, we end up with 6 tifs that broadly cover our area. But what if we wanted to confine our search area around the Colorado River, and reduce time spent on post processing of the data?  

An easier way would be to utilize GDAL's [Virtual Raster Datasets (VRT)](https://gdal.org/programs/gdalbuildvrt.html).

As part of our ingest process, OpenTopography builds VRTs for all its rasters to enable faster and more efficient data access.  With GDAL and VRTs, users can extract a specific area of interest and obtain a merged, clipped raster all in one operation.  For example, by using the vsicurl command with a targeted bounding box, we can extract a raster that covers the area of the Grand Canyon around the Colorado River.

In [4]:
!time gdal_translate --debug on /vsicurl/https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm.vrt -projwin -113.98 36.563 -111.777 35.7 -of COG -co\
 COMPRESS=deflate srtm_GC.tif
    

HTTP: libcurl/7.71.1 OpenSSL/1.1.1i zlib/1.2.11 libssh2/1.9.0 nghttp2/1.43.0
VSICURL: GetFileSize(https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm.vrt)=6236931  response_code=200
VSICURL: Downloading 0-16383 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm.vrt)...
VSICURL: Got response_code=206
VSICURL: Downloading 16384-6236930 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm.vrt)...
VSICURL: Got response_code=206
GDAL: GDALOpen(/vsicurl/https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm.vrt, this=0x7fb50d6a5560) succeeds as VRT.
Input file size is 1296001, 417601
GDAL: GDALDefaultOverviews::OverviewScan()
VSICURL: GetFileSize(https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm.vrt.ovr)=0  response_code=404
VSICURL: GetFileSize(https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm.vrt.OVR)=0  response_code=404
VSICURL: GetFileList(/vsicurl/https://opentopography.s3.sdsc.edu/raster/SRTM_GL1)
VSICURL

VSICURL: Download completed
VSICURL: Downloading 10200709-10694096 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm/N36W113.tif)...
VSICURL: Downloading 11291077-11496055 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm/N36W113.tif)...
VSICURL: Downloading 12100112-12582546 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm/N36W113.tif)...
VSICURL: Downloading 13388409-13623437 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm/N36W113.tif)...
VSICURL: Downloading 13895721-14066070 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm/N36W113.tif)...
VSICURL: Download completed
VSICURL: Downloading 8544377-8825203 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm/N36W114.tif)...
VSICURL: Downloading 9506275-10113860 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm/N36W114.tif)...
VSICURL: Downloading 11366319-11972049 (https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_

VSICURL: Download completed
..20GDAL: GDALClose(srtm_GC.tif.ovr.tmp, this=0x7fb50d60c9f0)
COG: Generating overviews of the imagery: end
COG: Generating final product: start
GDAL: GDALOpen(srtm_GC.tif.ovr.tmp, this=0x7fb50e89b7c0) succeeds as GTiff.
GTiff: ScanDirectories()
GTiff: Opened 1982x776 overview.
GTiff: Opened 991x388 overview.
GTiff: Opened 495x194 overview.
GTiff: ScanDirectories()
GDAL: GDALOverviewDataset(srtm_GC.tif.ovr.tmp, this=0x7fb50d848120) creation.
GDAL: GDALOverviewDataset(srtm_GC.tif.ovr.tmp, this=0x7fb50d35b2e0) creation.
GDAL: GDALOverviewDataset(srtm_GC.tif.ovr.tmp, this=0x7fb50d35b2e0) creation.
...30...40...50...60...70...80...90...100 - done.
GDAL: GDALClose(srtm_GC.tif.ovr.tmp, this=0x7fb50e89b7c0)
COG: Generating final product: end
GDAL: GDALClose(srtm_GC.tif, this=0x7fb509f74a90)
GDAL: GDALClose(/vsicurl/https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm.vrt, this=0x7fb50d6a5560)
GDAL: GDALClose(/vsicurl/https://opentopography.s3.sdsc.edu/r

---
So the above command runs pretty quickly, and by using GDAL version 3.2, which has native COG support, we are able to output a COG-formatted mosaic of SRTM data over our area of interest.  Note the '-co COMPRESS-deflate' command additionally compresses the output file using the 'deflate' algorithm to reduce the output filesize.  Here is the output from this command:

![GrandCanyonClipped](img/srtm_GConBasemap.png)

**NOTE:** Users can get the same data product by using [OpenTopography's API](https://opentopography.org/developers).  Users can [build the REST API call](https://portal.opentopography.org/apidocs/#/Public/getGlobalDem) and it will output the appropriate request to use in a curl command, or a http GET request.  Here is the URL request that is output for SRTM GL1 data over the GrandCanyon:

`https://portal.opentopography.org/API/globaldem?demtype=SRTMGL1&south=35.7&north=36.54&west=-114&east=-111.7&outputFormat=GTiff`


---
By using the VRTs we have now have a single, mosaiced geotiff that is much smaller in size than the original tiled images.  But we can take this further by using the power of COGs and other python libraries.  So, let's say we really only want data around the Colorado River.  To do this, we first need to get a vector representation of the Colorado River.  Luckily, there is a fantastic site called, [Natural Earth Data](https://www.naturalearthdata.com/) that has a wealth of vector and raster map data at a variety of scales.  For this example, we'll use the rivers and lakes centerlines file.  You can download the data from the [Natural Earth Data Download site](https://www.naturalearthdata.com/downloads/10m-physical-vectors/) or you can use wget.  You may need to install wget with a pip command:

In [6]:
!pip install wget

  Using cached wget-3.2-py3-none-any.whl


In [7]:
import wget
wget.download('https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_lake_centerlines.zip','rivers.zip')

'rivers.zip'

In [8]:
!unzip rivers.zip

Archive:  rivers.zip
  inflating: ne_10m_rivers_lake_centerlines.README.html  
 extracting: ne_10m_rivers_lake_centerlines.VERSION.txt  
 extracting: ne_10m_rivers_lake_centerlines.cpg  
  inflating: ne_10m_rivers_lake_centerlines.dbf  
  inflating: ne_10m_rivers_lake_centerlines.prj  
  inflating: ne_10m_rivers_lake_centerlines.shp  
  inflating: ne_10m_rivers_lake_centerlines.shx  


---
Now we have river data, but it is for the entire world. Next we'll clip the river dataset to our area of interest using [GDAL's vector toolkit](https://gdal.org/programs/ogr2ogr.html#ogr2ogr).  We also want to create a buffer around the river, but in order to do that we need to project the data.  As downloaded, the river data is in a Geographic Coordinate System, so we will need to output the data in a projected coordinate system in order to do the buffering operation.  Arizona is in UTM zone 12N, so we'll use [EPSG:32612](https://epsg.io/32612) for the output projection.  Using the ogr2ogr tool we can output a projected, clipped version of the river shapefile with the following command:

In [9]:
!ogr2ogr ColoradoRiver_AOI.shp ne_10m_rivers_lake_centerlines.shp -t_srs EPSG:32612 -spat -113.98 35.7 -111.777 36.563 -clipsrc spat_extent

Warning 1: One or several characters couldn't be converted correctly from UTF-8 to ISO-8859-1.  This warning will not be emitted anymore.


Next we need to create a buffer around the Colorado River.  We'll make a buffer that is 10km around either side so that we include the interesting features of the Grand Canyon.  With OGR you can pass in [SQLite commands](https://gdal.org/user/sql_sqlite_dialect.html).  In this example we will use the ST_Buffer command to create a 10km buffer around the Colorado River.

In [10]:
!ogr2ogr ColoradoRiver_AOI_10kmBuffer.shp ColoradoRiver_AOI.shp -dialect sqlite -sql "select ST_buffer(geometry,10000) as geometry from ColoradoRiver_AOI"

	- 'VirtualXPath'	[XML Path Language - XPath]
	- 'VirtualXPath'	[XML Path Language - XPath]


Since the river shapefile had segments, we should dissolve the buffer so that we end up with a single dissolved polygon that covers the buffer region of the river.  To do this, we can once again use the "ST_Union" command with sqlite:

In [11]:
!ogr2ogr ColoradoRiver_AOI_Buffer_Dissolved.shp ColoradoRiver_AOI_10kmBuffer.shp -dialect sqlite -sql "select ST_Union(geometry) as geometry from ColoradoRiver_AOI_10kmBuffer"

	- 'VirtualXPath'	[XML Path Language - XPath]
	- 'VirtualXPath'	[XML Path Language - XPath]


---
Since we buffered the data, it now extends outside our AOI.  We can clip it to the bounds, but the buffered shapefile is now in UTM 12N projected coordinates, so we need to find what the bounding box coordinates are in UTM 12N coordinates.  We can do this tranformation with OSR and OGR in the python API: 

In [12]:
from osgeo import ogr                                                                                                                                           
from osgeo import osr                                                                                                                                           
                                                                                                                                                                
# WGS84/Geographic                                                                                                                                              
InCRS = osr.SpatialReference()                                                                                                                                  
InCRS.ImportFromEPSG(4326)                                                                                                                                      
                                                                                                                                                                
# WGS84 UTM Zone 12N                                                                                                                                            
OutCRS = osr.SpatialReference()                                                                                                                                 
OutCRS.ImportFromEPSG(32612)                                                                                                                                    
                                                                                                                                                                
transform = osr.CoordinateTransformation(InCRS, OutCRS)                                                                                                         
                                                                                                                                                                
#NOTE in GDAL Versions > 3.0:  input order is lat,lon                                                                                                           
point_LL = ogr.CreateGeometryFromWkt("POINT (35.7 -113.98)")                                                                                                    
point_LL.Transform(transform)                                                                                                                                   
                                                                                                                                                                
                                                                                                                                                                
point_UR = ogr.CreateGeometryFromWkt("POINT (36.563 -111.777)")                                                                                                 
point_UR.Transform(transform)                                                                                                                                   
                                                                                                                                                                
print('Lower Left Corner in UTM X,Y is: '+point_LL.ExportToWkt())                                                                                               
print('Upper Right Corner in UTM X,Y is: '+point_UR.ExportToWkt())

Lower Left Corner in UTM X,Y is: POINT (230367.047752107 3954768.801329)
Upper Right Corner in UTM X,Y is: POINT (430471.347916117 4046677.17926595)


---
Now we can feed these UTM bounding coordinates back into a ogr to do the clipping:

In [13]:
!ogr2ogr ColoradoRiver_AOI_Final.shp ColoradoRiver_AOI_Buffer_Dissolved.shp -clipdst 230367.047752107 3954768.801329 430471.347916117 4046677.17926595

---
Now we have a final boundary that we can use to read in and clip the SRTM data

![ColoradoRiverBuffer](img/srtm_GC_wBuffer.png)


---
Because we are ultimately going to intersect this boundary with the SRTM VRT, they both need to be in the same coordinate system.  So, we need to re-project this final boundary back to Geographic Coordinates so that it is compatible with the SRTM dataset.  We can do this with a ogr2ogr command:



In [14]:
!ogr2ogr ColoradoRiver_AOI_Final_WGSLatLon.shp ColoradoRiver_AOI_Final.shp -s_srs "EPSG:32612" -t_srs "EPSG:4326"

Many applications can now take advantage of COGs.  [Rasterio](https://rasterio.readthedocs.io/en/latest/) is a python library that takes advantage of COGs ability to use the vsis3 and vsicurl file system handlers to perform HTTP reads in a more efficient way.  Ultimately, this benefit is passed on to the user by providing faster downloads that take up less disk space.  As an example, we will use [fiona](https://pypi.org/project/Fiona/) and [rasterio](https://rasterio.readthedocs.io/en/latest/) to download only the SRTM data that fits within our 10km Colorado River buffer. 

In [15]:
pip install fiona

  Using cached Fiona-1.8.18-cp39-cp39-macosx_10_9_x86_64.whl
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl (82 kB)
  Using cached munch-2.5.0-py2.py3-none-any.whl (10 kB)
  Using cached cligj-0.7.1-py3-none-any.whl (7.1 kB)
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install rasterio

  Using cached rasterio-1.2.0-cp39-cp39-macosx_10_9_x86_64.whl (20.5 MB)
  Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
  Using cached affine-2.3.0-py2.py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
import fiona                                                                                                                                                    
import rasterio                                                                                                                                                 
import rasterio.mask                                                                                                                                            
                                                                                                                                                                
inshp = 'ColoradoRiver_AOI_Final_WGSLatLon.shp'                                                         
fp = 'https://opentopography.s3.sdsc.edu/raster/SRTM_GL1/SRTM_GL1_srtm.vrt'                                                                                     

#read in the buffered shapefile using fiona
with fiona.open(inshp, "r") as shapefile:                                                                                                                       
    shapes = [feature["geometry"] for feature in shapefile]                                                                                                     
                                                                                                                                                                
#Using rasterio, read directly from the SMRT VRT of COGs, and crop
# data to the buffer polygon
with rasterio.open(fp) as src:                                                                                                                                  
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)                                                                                       
    out_meta = src.meta                                                                                                                                         

#set up the output parameters
out_meta.update({"driver": "GTiff",                                                                                                                             
                 "height": out_image.shape[1],                                                                                                                  
                 "width": out_image.shape[2],
                 "compress": "LZW",
                 "tiled":"True", 
                 "blockxsize":"256",
                 "blockysize":"256",
                 "transform": out_transform})                                                                                                                   
                                                                                                                                                                
#Write out just the SRTM data that is within the buffer polygon
with rasterio.open("GrandCanyon_masked.tif", "w", **out_meta) as dest:                                                                                                   
    dest.write(out_image)      

---
Using rasterio with OpenTopography's VRT of COGs, we are able to download only the portion of the SRTM data we need.  In this example, we downloaded only data that is within our 10 km polygon buffer of the portion of Colorado River around the Grand Canyon.  **Our final output raster is only 12MB, versus the 86MB of tiffs we started with at the beginning!!**  By putting all these tools together, users can create powerful scripts that automate data access, and ultimately reduce time on data downloads and post-processing.  

![Final Product](img/srtm_GC_clip2Buffer.png)

## Conclusion
Hopefully this tutorial has been helpful, and has demonstrated alternative methods to accessing cloud-based data.  We encourage users to explore these datasets with these powerful tools.  Opentopography has 4 global datasets that are available through our API and/or bulk access as described in this article:
* [SRTM](https://doi.org/10.5069/G9445JDF)
* [ALOS](https://doi.org/10.5069/G94M92HB)
* [STRM15+](https://doi.org/10.5069/G92R3PT9)



## Resources
Check out some of these excellent resources on GDAL, Rasterio, and COGs
* [A good introduction to COGs](https://www.cogeo.org/ )
* [Another good intro to COGs](https://medium.com/planet-stories/a-handy-introduction-to-cloud-optimized-geotiffs-1f2c9e716ec3)                    
* [Detailed info about the COG format in GDAL](https://trac.osgeo.org/gdal/wiki/CloudOptimizedGeoTIFF)                 
* [Python notebook on how to use Rasterio](https://gist.github.com/sgillies/7e5cd548110a5b4d45ac1a1d93cb17a3)
* [An excellent explanation of COG file structure](https://medium.com/element84/cloud-optimized-geotiff-vs-the-meta-raster-format-d24c1a77dc2e)
* [Reading COGs with Rasterio (and much more)](https://automating-gis-processes.github.io/CSC/notebooks/L5/read-cogs.html)
